In [2]:
import pandas as pd
import numpy as np
from os.path import join

In [3]:
path_raw = 'D:/Documents/Machine learning/Projet'


#file_members = 'members.csv'
#file_songs = 'songs.csv'
file_train = 'train.csv'
#file_test='test.csv'

In [4]:
#members = pd.read_csv(join(path_raw, file_members), error_bad_lines=False)
#songs = pd.read_csv(join(path_raw, file_songs), error_bad_lines=False)
train = pd.read_csv(join(path_raw, file_train), error_bad_lines=False)
#test = pd.read_csv(join(path_raw, file_test), error_bad_lines=False)

In [5]:
# Découpage de train 70% apprentissage, 30% validation
from sklearn.model_selection import train_test_split
traina, trainv = train_test_split(train, shuffle=True, test_size=0.3, stratify=train['target'])

In [6]:
trainv.head()

,msno,song_id,source_system_tab,source_screen_name,source_type,target
5459565,fVKfTvgh/kIY+jqOAW9JnNXh1H87NCu19rNu5rq1hzc=,LIoPcdYXg41ViflSfVXT0KRDH3cpVyD0P8KblR0cj9I=,discover,Online playlist more,song-based-playlist,0
311719,2B0Xu3dFx1ZNJxzMS4iV96uBqIrstiZgCliQrYwn6r4=,PuItp3JmRqAJvTIJnEXlFr0LHl2Lx+DBzq220QAvcqs=,my library,Local playlist more,local-library,1
4577100,j+li9F9QvOTn9/FvuRenIIX5QxGlu1XDda9EY8ifAbM=,pj23StjgrD8Kb0VqX+u/1/NF/hHqBbDTm0rklLuGfTM=,my library,Local playlist more,local-library,0
684513,eEw6KcTp5UVwsEOhIr7KA26RfIr1S0g015ykKrm/SA0=,/gazdvZBcr25cecyOc8wvL06bb0YbD/B4FN3zS11ow4=,listen with,Others profile more,listen-with,1
5605128,hTDMpkFGHNC7U/iJyECO0+6vZIAKT4oU7AlUM8iF5PI=,p/VaHNpHfpUIdHQ5lFCjrtq+0ZdatzB4HvtDZVjlx44=,my library,Local playlist more,local-library,1


In [5]:
# Pour chaque chanson, on récupère la liste de ses genres
genr=pd.DataFrame([str(element).split('|') for element in songs['genre_ids']])


In [6]:
genr.head()

,0,1,2,3,4,5,6,7
0,465,None,None,None,None,None,None,None
1,444,None,None,None,None,None,None,None
2,465,None,None,None,None,None,None,None
3,465,None,None,None,None,None,None,None
4,726,None,None,None,None,None,None,None


In [7]:
# chanson, liste de ses genres, id de la chanson
song_tr=pd.concat([songs[['song_id']], genr], axis=1)
song_tr['id']=song_tr.index
song_tr.head()

,song_id,0,1,2,3,4,5,6,7,id
0,CXoTN1eb7AI+DntdU1vbcwGRV4SCIDxZu+YD8JP8r4E=,465,None,None,None,None,None,None,None,0
1,o0kFgae9QtnYgRkVPqLJwa05zIhRlUjfF7O1tDw0ZDU=,444,None,None,None,None,None,None,None,1
2,DwVvVurfpuz+XPuFvucclVQEyPqcpUkHR0ne1RQzPs0=,465,None,None,None,None,None,None,None,2
3,dKMBWoZyScdxSkihKG+Vf47nc18N9q4m58+b4e7dSSE=,465,None,None,None,None,None,None,None,3
4,W3bqWd3T+VeHFzHAUfARgW9AvVRaF4N5Yzm4Mr6Eo/o=,726,None,None,None,None,None,None,None,4


In [8]:
# On rajoute l'id de l'utilisateur dans une colonne user_id
members['user_id']=members.index

In [9]:
members.head()

,msno,city,bd,gender,registered_via,registration_init_time,expiration_date,user_id
0,XQxgAYj3klVKjR3oxPPXYYFp4soD4TuBghkhMTD4oTw=,1,0,NaN,7,20110820,20170920,0
1,UizsfmJb9mV54qE9hCYyU07Va97c0lCRLEQX3ae+ztM=,1,0,NaN,7,20150628,20170622,1
2,D8nEhsIOBSoE6VthTaqDX8U6lqjJ7dLdr72mOyLya2A=,1,0,NaN,4,20160411,20170712,2
3,mCuD+tZ1hERA/o5GPqk38e041J8ZsBaLcu7nGoIIvhI=,1,0,NaN,9,20150906,20150907,3
4,q4HRBfVSssAFS9iRfxWrohxuk9kCYMKjHOEagUMV6rQ=,1,0,NaN,4,20170126,20170613,4


In [10]:
def remove_none_elements_from_list(liste):
    return [e for e in liste if e != None ]
def pos_genre(group):
    return pd.DataFrame({'genre':pd.get_dummies(
        remove_none_elements_from_list(group[list(range(8))].values.flatten())).sum()})
def neg_genre(group):
    return pd.DataFrame({'genre':-pd.get_dummies(
        remove_none_elements_from_list(group[list(range(8))].values.flatten())).sum()})
def transf_to_score_genre(part_train,binar):# prend une partie de train et 0 ou 1 et retourne un tableau avec 
    # les scores des genres pour chaque utilisateur positif(chanson appréciée) ou négatif
    X=part_train[['msno','song_id','target']].merge(song_tr, on='song_id').merge(members[['msno','user_id']], on='msno')
    X = X.drop(['msno','song_id'],1)
    if binar==0:
        gen=X.where(X['target']==binar).groupby('user_id').apply(neg_genre)
    else:
        gen=X.where(X['target']==binar).groupby('user_id').apply(pos_genre)
    liste=[]
    for index in np.unique([val1 for (val1,val2) in gen.index]):
        liste.append(pd.DataFrame(gen['genre'][index]).transpose())
    a=pd.concat(liste)
    a=a.drop(['nan'],1)
    a.index=np.unique([val1 for (val1,val2) in gen.index])
    a=a.fillna(0)
    return a

In [14]:
pos=transf_to_score_genre(traina,1)

In [15]:
pos.head()

,1000,1007,1011,1019,102,1026,1033,1040,1047,1054,...,94,940,947,95,958,965,972,979,986,993
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,4.0,0.0,2.0,0.0,0.0,0.0
9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
writer = pd.ExcelWriter('score_traina_pos_genre_utilisateur.xlsx')
pos.to_excel(writer,'Sheet 1')
writer.save()

In [17]:
trainv.to_csv('valid')